# Conjunto Dados NFT's Comuns - ML Autoencoders

In [1]:
pip install google-cloud-bigquery db-dtypes pandas tensorflow scikit-learn

In [2]:
# importar pacotes
import os
import requests
import numpy as np
import glob
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from google.cloud import bigquery
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.svm import OneClassSVM
from sklearn.metrics import silhouette_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.cluster import DBSCAN

from tensorflow import keras
from tensorflow.keras import layers, models

## Conjunto de Dados - Data Sets (NFT's Blue Chip's e NFT's Comuns)

Arquivos CSV criados a partir da extração de dados no Google BigQuery contém as seguintes informações:

* BLOCK_TIMESTAMP: timestamp do bloco em que a transferência foi registrada.
* FROM_ADDRESS: endereço da carteira que enviou os tokens.
* TO_ADDRESS: endereço da carteira que recebeu os tokens.
* VALUE: valor da transferência em wei, a menor unidade de ether (a criptomoeda da Ethereum).
* TRANSACTION_HASH: hash da transação na blockchain Ethereum.
* NONCE: número de sequência da transação na carteira que a criou.
* FROM_ADDRESS_BLOCKCHAIN: endereço da carteira que enviou a transação na blockchain Ethereum.
* TO_ADDRESS_BLOCKCHAIN: endereço da carteira que recebeu a transação na blockchain Ethereum.
* GAS: quantidade de gas (unidade de custo de processamento na Ethereum) usada pela transação.
* RECEIPT_GAS_USED: quantidade de gas usada para executar a transferência de tokens.

In [3]:
def lista_df():
  # Lista vazia para armazenar os dataframes
  df_list = []

  # Localizar todos os arquivos CSV no diretório atual
  for filename in glob.glob("Z:/Python/Data/Blue_Chips/*.csv"):
      # Ler o arquivo CSV em um dataframe
      df = pd.read_csv(filename)
      # Adicionar o dataframe à lista
      df_list.append(df)

  return df_list

In [4]:
def lista_comum_df():
  # lista vazia para armazenar os dataframes
  df_comum_list = []

  # Localizar todos os arquivos CSV no diretório atual
  for filename in glob.glob("Z:/Python/Data/Comuns/*.csv"):

      # Ler o arquivo CSV em um dataframe
      df = pd.read_csv(filename)
      # Adicionar o dataframe à lista
      df_comum_list.append(df)

  return df_comum_list

In [5]:
def process_dataframe(df):
    # Convertendo a coluna 'BLOCK_TIMESTAMP' e 'BLOCK_TIMESTAMP_BLUE' para o tipo datetime
    df['BLOCK_TIMESTAMP'] = pd.to_datetime(df['BLOCK_TIMESTAMP'])

    # Criando uma nova coluna com a data (sem o horário) dos blocos
    df['BLOCK_DATE'] = df['BLOCK_TIMESTAMP'].dt.date

    # Convertendo a coluna 'BLOCK_DATE' para o tipo datetime
    df['BLOCK_DATE'] = pd.to_datetime(df['BLOCK_DATE'])

In [6]:
list_df_blue = []
list_df_blue = lista_df()
len(list_df_blue)

20

In [7]:
list_df_comum = []
list_df_comum = lista_comum_df()
len(list_df_comum)

24

In [8]:
# Loop para iterar sobre as list_df_blue, list_df_comum
for idx, df_group in enumerate([list_df_blue, list_df_comum]):
    if idx == 0:
        group_label = "Blue Chip"
    else:
        group_label = "Comum"

    for i, df_aux in enumerate(df_group):
        print(f"\nData Frame {group_label} {i+1}")
        print("Informações:")
        df_aux.info()
        print("\n5 primeiros registros:")
        print(df_aux.head())



Data Frame Blue Chip 1
Informações:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98206 entries, 0 to 98205
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ADDRESS_BLUE             98206 non-null  object
 1   BLOCK_TIMESTAMP_BLUE     98206 non-null  object
 2   BLOCK_TIMESTAMP          98206 non-null  object
 3   FROM_ADDRESS             98206 non-null  object
 4   TO_ADDRESS               98206 non-null  object
 5   VALUE                    98206 non-null  int64 
 6   TRANSACTION_HASH         98206 non-null  object
 7   NONCE                    98206 non-null  int64 
 8   FROM_ADDRESS_BLOCKCHAIN  98206 non-null  object
 9   TO_ADDRESS_BLOCKCHAIN    98206 non-null  object
 10  GAS                      98206 non-null  int64 
 11  RECEIPT_GAS_USED         98206 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 9.0+ MB

5 primeiros registros:
                                 ADDRE

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13778 entries, 0 to 13777
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ADDRESS_BLUE             13778 non-null  object
 1   BLOCK_TIMESTAMP_BLUE     13778 non-null  object
 2   BLOCK_TIMESTAMP          13778 non-null  object
 3   FROM_ADDRESS             13778 non-null  object
 4   TO_ADDRESS               13778 non-null  object
 5   VALUE                    13778 non-null  int64 
 6   TRANSACTION_HASH         13778 non-null  object
 7   NONCE                    13778 non-null  int64 
 8   FROM_ADDRESS_BLOCKCHAIN  13778 non-null  object
 9   TO_ADDRESS_BLOCKCHAIN    13778 non-null  object
 10  GAS                      13778 non-null  int64 
 11  RECEIPT_GAS_USED         13778 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 1.3+ MB

5 primeiros registros:
                                 ADDRESS_BLUE     BLOCK_TIMESTAMP_BLUE  \
0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117013 entries, 0 to 117012
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   ADDRESS_BLUE             117013 non-null  object
 1   BLOCK_TIMESTAMP_BLUE     117013 non-null  object
 2   BLOCK_TIMESTAMP          117013 non-null  object
 3   FROM_ADDRESS             117013 non-null  object
 4   TO_ADDRESS               117013 non-null  object
 5   VALUE                    117013 non-null  int64 
 6   TRANSACTION_HASH         117013 non-null  object
 7   NONCE                    117013 non-null  int64 
 8   FROM_ADDRESS_BLOCKCHAIN  117013 non-null  object
 9   TO_ADDRESS_BLOCKCHAIN    117013 non-null  object
 10  GAS                      117013 non-null  int64 
 11  RECEIPT_GAS_USED         117013 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 10.7+ MB

5 primeiros registros:
                                 ADDRESS_BLUE     BLOCK_TI

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66167 entries, 0 to 66166
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ADDRESS_BLUE             66167 non-null  object
 1   BLOCK_TIMESTAMP_BLUE     66167 non-null  object
 2   BLOCK_TIMESTAMP          66167 non-null  object
 3   FROM_ADDRESS             66167 non-null  object
 4   TO_ADDRESS               66167 non-null  object
 5   VALUE                    66167 non-null  int64 
 6   TRANSACTION_HASH         66167 non-null  object
 7   NONCE                    66167 non-null  int64 
 8   FROM_ADDRESS_BLOCKCHAIN  66167 non-null  object
 9   TO_ADDRESS_BLOCKCHAIN    66167 non-null  object
 10  GAS                      66167 non-null  int64 
 11  RECEIPT_GAS_USED         66167 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 6.1+ MB

5 primeiros registros:
                                 ADDRESS_BLUE     BLOCK_TIMESTAMP_BLUE  \
0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118958 entries, 0 to 118957
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   ADDRESS_BLUE             118958 non-null  object
 1   BLOCK_TIMESTAMP_BLUE     118958 non-null  object
 2   BLOCK_TIMESTAMP          118958 non-null  object
 3   FROM_ADDRESS             118958 non-null  object
 4   TO_ADDRESS               118958 non-null  object
 5   VALUE                    118958 non-null  int64 
 6   TRANSACTION_HASH         118958 non-null  object
 7   NONCE                    118958 non-null  int64 
 8   FROM_ADDRESS_BLOCKCHAIN  118958 non-null  object
 9   TO_ADDRESS_BLOCKCHAIN    118958 non-null  object
 10  GAS                      118958 non-null  int64 
 11  RECEIPT_GAS_USED         118958 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 10.9+ MB

5 primeiros registros:
                                 ADDRESS_BLUE     BLOCK_TI

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178767 entries, 0 to 178766
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   ADDRESS_BLUE             178767 non-null  object
 1   BLOCK_TIMESTAMP_BLUE     178767 non-null  object
 2   BLOCK_TIMESTAMP          178767 non-null  object
 3   FROM_ADDRESS             178767 non-null  object
 4   TO_ADDRESS               178767 non-null  object
 5   VALUE                    178767 non-null  int64 
 6   TRANSACTION_HASH         178767 non-null  object
 7   NONCE                    178767 non-null  int64 
 8   FROM_ADDRESS_BLOCKCHAIN  178767 non-null  object
 9   TO_ADDRESS_BLOCKCHAIN    178755 non-null  object
 10  GAS                      178767 non-null  int64 
 11  RECEIPT_GAS_USED         178767 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 16.4+ MB

5 primeiros registros:
                                 ADDRESS_BLUE     BLOCK_TI

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2481 entries, 0 to 2480
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   BLOCK_TIMESTAMP          2481 non-null   object
 1   FROM_ADDRESS             2481 non-null   object
 2   TO_ADDRESS               2481 non-null   object
 3   VALUE                    2481 non-null   int64 
 4   TRANSACTION_HASH         2481 non-null   object
 5   NONCE                    2481 non-null   int64 
 6   FROM_ADDRESS_BLOCKCHAIN  2481 non-null   object
 7   TO_ADDRESS_BLOCKCHAIN    2481 non-null   object
 8   GAS                      2481 non-null   int64 
 9   RECEIPT_GAS_USED         2481 non-null   int64 
dtypes: int64(4), object(6)
memory usage: 194.0+ KB

5 primeiros registros:
                  BLOCK_TIMESTAMP                                FROM_ADDRESS  \
0  2023-03-15 12:53:23.000000 UTC  0x0000000000000000000000000000000000000000   
1  2023-03-15 12:53

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18044 entries, 0 to 18043
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   BLOCK_TIMESTAMP          18044 non-null  object
 1   FROM_ADDRESS             18044 non-null  object
 2   TO_ADDRESS               18044 non-null  object
 3   VALUE                    18044 non-null  int64 
 4   TRANSACTION_HASH         18044 non-null  object
 5   NONCE                    18044 non-null  int64 
 6   FROM_ADDRESS_BLOCKCHAIN  18044 non-null  object
 7   TO_ADDRESS_BLOCKCHAIN    18044 non-null  object
 8   GAS                      18044 non-null  int64 
 9   RECEIPT_GAS_USED         18044 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 1.4+ MB

5 primeiros registros:
                  BLOCK_TIMESTAMP                                FROM_ADDRESS  \
0  2023-03-07 19:50:59.000000 UTC  0x0000000000000000000000000000000000000000   
1  2023-03-07 19:50

In [9]:
# Processando a lista de DataFrames list_df_blue
df_list_blue = []
for df_blue in list_df_blue:
    process_dataframe(df_blue)
    df_list_blue.append(df_blue)

# Processando a lista de DataFrames list_df_comum
df_list_comum = []
for df_comum in list_df_comum:
    process_dataframe(df_comum)
    df_list_comum.append(df_comum)


In [10]:
list_grouped_df_blue = []
list_grouped_df_comum = []

# Loop para agrupar os DataFrames e criar as listas list_grouped_df_blue e list_grouped_df_comum
for dataframe_list, grouped_list in zip([df_list_blue, df_list_comum], [list_grouped_df_blue, list_grouped_df_comum]):
    for df in dataframe_list:
        grouped_df = df.groupby('BLOCK_DATE')
        grouped_list.append(grouped_df)

# Loop para imprimir os tamanhos dos grupos para ambos os DataFrames agrupados
for label, grouped_list in [("Blue Chip", list_grouped_df_blue), ("Comum", list_grouped_df_comum)]:
    print(f"\n{label} DataFrames:")
    
    for i, grouped_df in enumerate(grouped_list):
        print(f"\ngrouped_df_{label} {i+1}")
        
        # Use a função size() para obter o tamanho de cada grupo
        group_sizes = grouped_df.size()
        
        # Imprima os tamanhos dos grupos
        print(group_sizes)



Blue Chip DataFrames:

grouped_df_Blue Chip 1
BLOCK_DATE
2022-01-06    8231
2022-01-07    2905
2022-01-08    1250
2022-01-09     618
2022-01-10     315
              ... 
2023-03-10     102
2023-03-11      36
2023-03-12     150
2023-03-13      64
2023-03-14       5
Length: 433, dtype: int64

grouped_df_Blue Chip 2
BLOCK_DATE
2022-04-29     15000
2022-04-30        81
2022-05-01    131388
2022-05-02     16965
2022-05-03      6447
               ...  
2023-03-10      1182
2023-03-11      1057
2023-03-12       723
2023-03-13       857
2023-03-14        48
Length: 320, dtype: int64

grouped_df_Blue Chip 3
BLOCK_DATE
2021-07-22    9654
2021-07-23     515
2021-07-24     232
2021-07-25     256
2021-07-26     166
              ... 
2023-03-10     206
2023-03-11     152
2023-03-12     101
2023-03-13     174
2023-03-14      22
Length: 601, dtype: int64

grouped_df_Blue Chip 4
BLOCK_DATE
2022-10-31    3734
2022-11-01     165
2022-11-02      94
2022-11-03     110
2022-11-04      20
              .

In [11]:
ts_nft_comum = []  # Lista para armazenar os DataFrames criados a partir de list_grouped_df_comum
ts_nft_blue = []   # Lista para armazenar os DataFrames criados a partir de list_grouped_df_blue

# Loop para iterar sobre list_grouped_df_comum e list_grouped_df_blue
for label, grouped_list, ts_nft_list in [("Comum", list_grouped_df_comum, ts_nft_comum), ("Blue Chip", list_grouped_df_blue, ts_nft_blue)]:
    print(f"\n{label} DataFrames:")
    
    for i, ts in enumerate(grouped_list):
        print(f"\ngrouped_df_{label} {i+1}")
        
        # Criar um novo DataFrame vazio para cada grupo
        df = pd.DataFrame()
        
        # Adicionar a coluna 'BLOCK_DATE' ao DataFrame
        df['BLOCK_DATE'] = ts['BLOCK_DATE'].mean()
        
        # Número de transações diárias
        df['QTDE_TRANSACOES_DIA'] = ts['BLOCK_DATE'].count()
        
        # Média de GAS diário
        df['MEDIA_GAS_DIA'] = ts['GAS'].mean()
        
        # Média de GAS_LIMIT diário
        df['MEDIA_GAS_LIMIT_DIA'] = ts['RECEIPT_GAS_USED'].mean()
        
        # Quantidade de compradores únicos diários
        df['QTDE_COMPRADORES_UNICOS_DIA'] = ts['FROM_ADDRESS'].nunique()
        
        # Quantidade de vendedores únicos diários
        df['QTDE_VENDEDORES_UNICOS_DIA'] = ts['TO_ADDRESS'].nunique()
        
        # Cálculo de NEW_HOLDER
        df['NEW_HOLDER'] = (ts['FROM_ADDRESS'].nunique() + ts['TO_ADDRESS'].nunique()) / ts['BLOCK_DATE'].count()
        
        # Transforma a coluna 'BLOCK_DATE' em índice temporal
        df.set_index('BLOCK_DATE', inplace=True)
        
        # Adicionar o DataFrame criado à lista correspondente
        ts_nft_list.append(df)



Comum DataFrames:

grouped_df_Comum 1

grouped_df_Comum 2

grouped_df_Comum 3

grouped_df_Comum 4

grouped_df_Comum 5

grouped_df_Comum 6

grouped_df_Comum 7

grouped_df_Comum 8

grouped_df_Comum 9

grouped_df_Comum 10

grouped_df_Comum 11

grouped_df_Comum 12

grouped_df_Comum 13

grouped_df_Comum 14

grouped_df_Comum 15

grouped_df_Comum 16

grouped_df_Comum 17

grouped_df_Comum 18

grouped_df_Comum 19

grouped_df_Comum 20

grouped_df_Comum 21

grouped_df_Comum 22

grouped_df_Comum 23

grouped_df_Comum 24

Blue Chip DataFrames:

grouped_df_Blue Chip 1

grouped_df_Blue Chip 2

grouped_df_Blue Chip 3

grouped_df_Blue Chip 4

grouped_df_Blue Chip 5

grouped_df_Blue Chip 6

grouped_df_Blue Chip 7

grouped_df_Blue Chip 8

grouped_df_Blue Chip 9

grouped_df_Blue Chip 10

grouped_df_Blue Chip 11

grouped_df_Blue Chip 12

grouped_df_Blue Chip 13

grouped_df_Blue Chip 14

grouped_df_Blue Chip 15

grouped_df_Blue Chip 16

grouped_df_Blue Chip 17

grouped_df_Blue Chip 18

grouped_df_Blue Chip 

In [12]:
# Verifique os 5 primeiros registros de ts_nft_comum
print("5 primeiros registros de ts_nft_comum:")
for i, df in enumerate(ts_nft_comum):
    print(f"\nDataFrame {i+1} (ts_nft_comum):")
    print(df.head(5))

# Verifique os 5 primeiros registros de ts_nft_blue
print("\n5 primeiros registros de ts_nft_blue:")
for i, df in enumerate(ts_nft_blue):
    print(f"\nDataFrame {i+1} (ts_nft_blue):")
    print(df.head(5))


5 primeiros registros de ts_nft_comum:

DataFrame 1 (ts_nft_comum):
            QTDE_TRANSACOES_DIA  MEDIA_GAS_DIA  MEDIA_GAS_LIMIT_DIA  \
BLOCK_DATE                                                            
2023-03-04                27781   1.802284e+06         1.315986e+06   
2023-03-05                 7242   2.136430e+06         1.561812e+06   
2023-03-06                  816   2.605338e+06         1.983021e+06   
2023-03-07                  140   7.537273e+05         5.746781e+05   
2023-03-08                   41   4.586475e+05         3.440642e+05   

            QTDE_COMPRADORES_UNICOS_DIA  QTDE_VENDEDORES_UNICOS_DIA  \
BLOCK_DATE                                                            
2023-03-04                         2852                        4453   
2023-03-05                         1475                        1412   
2023-03-06                          189                         163   
2023-03-07                           54                          23   
2023-03-

In [13]:
# Função para normalizar uma lista de DataFrames
def normalize_dataframes(dataframe_list):
    # Inicialize o MinMaxScaler
    scaler = MinMaxScaler()

    for idx, ts in enumerate(dataframe_list):
        # Selecione apenas as colunas numéricas para normalização
        columns_to_normalize = [
            'QTDE_TRANSACOES_DIA',
            'MEDIA_GAS_DIA',
            'MEDIA_GAS_LIMIT_DIA',
            'QTDE_COMPRADORES_UNICOS_DIA',
            'QTDE_VENDEDORES_UNICOS_DIA',
            'NEW_HOLDER'
        ]

        # Aplique o Min-Max Scaling às colunas selecionadas
        dataframe_list[idx][columns_to_normalize] = scaler.fit_transform(ts[columns_to_normalize])



In [14]:
# Normalizar ts_nft_comum
normalize_dataframes(ts_nft_comum)

# Normalizar ts_nft_blue
normalize_dataframes(ts_nft_blue)


In [15]:
# Função para imprimir informações e os 5 primeiros registros de uma lista de DataFrames
def print_info_and_head(dataframe_list):
    for i, df in enumerate(dataframe_list):
        print(f"\nDataFrame {i+1}:")
        print("Informações após a normalização:")
        print(df.info())
        print("5 primeiros registros após a normalização:")
        print(df.head())

# Imprimir informações e os 5 primeiros registros de ts_nft_blue após a normalização
print("Informações e 5 primeiros registros de ts_nft_blue após a normalização:")
print_info_and_head(ts_nft_blue)

print("\n=======================================================================================\n")


# Imprimir informações e os 5 primeiros registros de ts_nft_comum após a normalização
print("Informações e 5 primeiros registros de ts_nft_comum após a normalização:")
print_info_and_head(ts_nft_comum)



Informações e 5 primeiros registros de ts_nft_blue após a normalização:

DataFrame 1:
Informações após a normalização:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 433 entries, 2022-01-06 00:00:00 to 2023-03-14 00:00:00
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   QTDE_TRANSACOES_DIA          433 non-null    float64
 1   MEDIA_GAS_DIA                433 non-null    float64
 2   MEDIA_GAS_LIMIT_DIA          433 non-null    float64
 3   QTDE_COMPRADORES_UNICOS_DIA  433 non-null    float64
 4   QTDE_VENDEDORES_UNICOS_DIA   433 non-null    float64
 5   NEW_HOLDER                   433 non-null    float64
dtypes: float64(6)
memory usage: 23.7 KB
None
5 primeiros registros após a normalização:
            QTDE_TRANSACOES_DIA  MEDIA_GAS_DIA  MEDIA_GAS_LIMIT_DIA  \
BLOCK_DATE                                                            
2022-01-06             0.733482       0.06603

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1785 entries, 2017-06-23 00:00:00 to 2023-03-18 00:00:00
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   QTDE_TRANSACOES_DIA          1785 non-null   float64
 1   MEDIA_GAS_DIA                1785 non-null   float64
 2   MEDIA_GAS_LIMIT_DIA          1785 non-null   float64
 3   QTDE_COMPRADORES_UNICOS_DIA  1785 non-null   float64
 4   QTDE_VENDEDORES_UNICOS_DIA   1785 non-null   float64
 5   NEW_HOLDER                   1785 non-null   float64
dtypes: float64(6)
memory usage: 97.6 KB
None
5 primeiros registros após a normalização:
            QTDE_TRANSACOES_DIA  MEDIA_GAS_DIA  MEDIA_GAS_LIMIT_DIA  \
BLOCK_DATE                                                            
2017-06-23             0.023649       0.007812             0.004646   
2017-06-24             0.030405       0.007512             0.004957   
2017-06-25           

                               QTDE_TRANSACOES_DIA  MEDIA_GAS_DIA  \
BLOCK_DATE                                                          
2023-03-24 00:00:00.000000000             0.255256       0.000000   
2023-03-25 00:00:00.000000000             1.000000       0.070658   
2023-03-26 00:00:00.000000256             0.196152       0.665472   
2023-03-26 23:59:59.999999744             0.085879       0.439331   
2023-03-28 00:00:00.000000000             0.064062       1.000000   

                               MEDIA_GAS_LIMIT_DIA  \
BLOCK_DATE                                           
2023-03-24 00:00:00.000000000             0.000000   
2023-03-25 00:00:00.000000000             0.090975   
2023-03-26 00:00:00.000000256             0.747666   
2023-03-26 23:59:59.999999744             0.478757   
2023-03-28 00:00:00.000000000             1.000000   

                               QTDE_COMPRADORES_UNICOS_DIA  \
BLOCK_DATE                                                   
2023-03-24 00

## Divisão do dados do Dataframe Blue Chips normalizados 

Divisão do DataFrame (normalized_dataset) em conjuntos de treinamento, validação e teste usando a função train_test_split do scikit-learn.

* train_test_split(normalized_dataset, test_size=0.4, random_state=42): Divisão do DataFrame **normalized_dataset** em dois conjuntos, o conjunto de **treinamento (train_data)** e o conjunto **temporário (temp_data)**. O hiperparâmetro test_size=0.4 define que 40% dos dados serão usados para o conjunto de teste e, consequentemente, 60% dos dados restantes serão usados para o conjunto de treinamento. O random_state=42 é usado para garantir que a divisão dos dados seja reproduzível.

* train_test_split(temp_data, test_size=0.5, random_state=42): Divisão do conjunto **temporário (temp_data)** em conjuntos de **validação (validation_data)** e **teste (test_data)**. O hiperparâmetro test_size=0.5 define que 50% dos dados serão usados para o conjunto de teste e, portanto, os outros 50% serão usados para o conjunto de validação. Novamente, o random_state=42 é usado para garantir que a divisão seja reproduzível.

Após essas operações, teremmos os seguintes conjuntos de dados:

* **train_data**: Conjunto de treinamento, que representa 60% dos dados originais.
* **validation_data**: Conjunto de validação, que representa 20% dos dados originais.
* **test_data**: Conjunto de teste, que também representa 20% dos dados originais.

Esses conjuntos podem ser usados para **treinar e testar** os modelos de detecção de anomalias (por exemplo, LOF) e, posteriormente, avaliar o desempenho do modelo usando as métricas adequadas, como **precisão, recall, F1-score** e outras métricas relevantes para o problema específico de detecção de anomalias.

In [16]:
# Dividir cada DataFrame em ts_nft_blue em conjuntos de treinamento, validação e teste
train_data_blue = []
validation_data_blue = []
test_data_blue = []

for df in ts_nft_blue:
    # Dividir o DataFrame em treinamento (60%), validação (20%) e teste (20%)
    train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42)
    validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
    
    # Adicionar os DataFrames resultantes às listas correspondentes
    train_data_blue.append(train_df)
    validation_data_blue.append(validation_df)
    test_data_blue.append(test_df)


In [17]:
# Inicialize listas vazias para armazenar os valores das métricas para mapa de calor (heatmap)
precision_scores_heatmap = []
recall_scores_heatmap = []
f1_scores_heatmap = []
accuracy_scores_heatmap = []

# Inicialize listas vazias para armazenar os valores das métricas
precision_scores = []
recall_scores = []
f1_scores = []
accuracy_scores = []

# Inicialize lista vazia para armazenar a existencia de TP para anomalia
true_positive_anomalies = []


In [18]:
# Função para calcular e retornar a média de precisão, recall e F1-score
def calculate_metrics(predictions, true_labels):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    return accuracy, precision, recall, f1

In [19]:
# Função para calcular e imprimir a matriz de confusão
def print_confusion_matrix(true_labels, predictions):
    confusion = confusion_matrix(true_labels, predictions)
    print("Matriz de Confusão:")
    print(confusion)

In [20]:
def encontrar_tp(confusion_matrix_val):
    """
    Verifique se um Verdadeiro Positivo (TP) ocorreu em uma única matriz de confusão. Posição no array da matriz para TP é [0,0], ou seja, primeira linha e primeira coluna

    Args:
        confusion_matrix_val (array): Matriz de confusão para um único conjunto de teste.

    Returns:
        bool: True se um TP for encontrado no conjunto de teste, False caso contrário.
    """
    tp_found = confusion_matrix_val[0, 0] > 0
    return tp_found



## **AUTOENCODERS**

### Métricas no conjunto de validação (validation_data_blue) e teste (test_data_blue) de NFT Blue Chips ###

In [21]:
# Construir o Autoencoder
input_dim = train_data_blue[0].shape[1]
encoding_dim = 32

input_layer = layers.Input(shape=(input_dim,))
encoder = layers.Dense(encoding_dim, activation='relu')(input_layer)
decoder = layers.Dense(input_dim, activation='sigmoid')(encoder)
autoencoder = models.Model(inputs=input_layer, outputs=decoder)

autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Treinar o modelo com os dados de treinamento
for train_df in train_data_blue:
    autoencoder.fit(train_df, train_df, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

# Definir um threshold para classificação de anomalias
threshold = 0.1  # Você pode ajustar este valor conforme necessário

for i, validation_df in enumerate(validation_data_blue):
    validation_predictions = autoencoder.predict(validation_df)
    distances_valid = np.linalg.norm(validation_df - validation_predictions, axis=1)

    # Fazer as previsões binárias no conjunto de validação usando o limiar
    valid_pred_binary = [1 if dist > threshold else 0 for dist in distances_valid]

    # Selecionar apenas a coluna com as classes de anomalia no conjunto de validação
    valid_labels = validation_df.iloc[:, 0].values
    # Transformar as classes de anomalia em 0 (não anomalia) e 1 (anomalia)
    valid_labels_binary = [1 if label != 0 else 0 for label in valid_labels]

    # Calcular métricas para o conjunto de validação
    accuracy_valid = accuracy_score(valid_labels_binary, valid_pred_binary)
    recall_valid = recall_score(valid_labels_binary, valid_pred_binary)
    precision_valid = precision_score(valid_labels_binary, valid_pred_binary)
    f1_score_valid = f1_score(valid_labels_binary, valid_pred_binary)
    
    # Armazenar os valores das métricas nas listas
    accuracy_scores.append(accuracy_valid)
    precision_scores.append(precision_valid)
    recall_scores.append(recall_valid)
    f1_scores.append(f1_score_valid)

    # Calcular a matriz de confusão para o conjunto de validação
    confusion_valid = confusion_matrix(valid_labels_binary, valid_pred_binary)

    # Imprimir a matriz de confusão
    print("Matriz de Confusão para o conjunto de validação " + str(i+1) + ":")
    print(confusion_valid) 
    
    # Imprimir as métricas
    print("\nMétricas para o conjunto de validação " + str(i+1) + ":")
    print("Acurácia:", accuracy_valid)
    print("Precisão:", precision_valid)
    print("Recall:", recall_valid)
    print("F1-score:", f1_score_valid)
    print("\n*******\n")
    
# Calcular as médias das métricas após o loop
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
average_precision = sum(precision_scores) / len(precision_scores)
average_recall = sum(recall_scores) / len(recall_scores)
average_f1 = sum(f1_scores) / len(f1_scores)

# Imprimir as médias das métricas
print("Média das Métricas:")
print("Acurácia:", average_accuracy)
print("Precisão:", average_precision)
print("Recall:", average_recall)
print("F1-score:", average_f1)

for lista in (accuracy_scores, precision_scores, recall_scores, f1_scores):
    while lista:
        lista.pop()

for i, test_df in enumerate(test_data_blue):
    test_predictions = autoencoder.predict(test_df)
    distances_test = np.linalg.norm(test_df - test_predictions, axis=1)

    # Fazer as previsões binárias no conjunto de teste usando o limiar
    test_pred_binary = [1 if dist > threshold else 0 for dist in distances_test]

    # Selecionar apenas a coluna com as classes de anomalia no conjunto de teste
    test_labels = test_df.iloc[:, 0].values
    # Transformar as classes de anomalia em 0 (não anomalia) e 1 (anomalia)
    test_labels_binary = [1 if label != 0 else 0 for label in test_labels]

    # Calcular métricas para o conjunto de validação
    accuracy_test = accuracy_score(test_labels_binary, test_pred_binary)
    precision_test = precision_score(test_labels_binary, test_pred_binary)
    recall_test = recall_score(test_labels_binary, test_pred_binary)
    f1_score_test = f1_score(test_labels_binary, test_pred_binary)
    
    # Armazenar os valores das métricas nas listas
    accuracy_scores.append(accuracy_test)
    precision_scores.append(precision_test)
    recall_scores.append(recall_test)
    f1_scores.append(f1_score_test)

    # Calcular a matriz de confusão para o conjunto de validação
    confusion_test = confusion_matrix(test_labels_binary, test_pred_binary)

    # Imprimir a matriz de confusão
    print("Matriz de Confusão para o conjunto de teste " + str(i+1) + ":")
    print(confusion_test) 
    
    # Imprimir as métricas
    print("\nMétricas para o conjunto de teste " + str(i+1) + ":")
    print("Precisão:", precision_test)
    print("Recall:", recall_test)
    print("F1-score:", f1_score_test)
    print("\n*******\n")
    
# Calcular as médias das métricas após o loop
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
average_precision = sum(precision_scores) / len(precision_scores)
average_recall = sum(recall_scores) / len(recall_scores)
average_f1 = sum(f1_scores) / len(f1_scores)

# Imprimir as médias das métricas
print("Média das Métricas:")
print("Acurácia:", average_accuracy)
print("Precisão:", average_precision)
print("Recall:", average_recall)
print("F1-score:", average_f1)

for lista in (accuracy_scores, precision_scores, recall_scores, f1_scores):
    while lista:
        lista.pop()

3/3 [==============================] - 0s 0s/step
Matriz de Confusão para o conjunto de validação 1:
[[ 0  0]
 [83  4]]

Métricas para o conjunto de validação 1:
Acurácia: 0.04597701149425287
Precisão: 1.0
Recall: 0.04597701149425287
F1-score: 0.08791208791208792

*******

2/2 [==============================] - 0s 16ms/step
Matriz de Confusão para o conjunto de validação 2:
[[ 0  0]
 [63  1]]

Métricas para o conjunto de validação 2:
Acurácia: 0.015625
Precisão: 1.0
Recall: 0.015625
F1-score: 0.03076923076923077

*******

4/4 [==============================] - 0s 0s/step
Matriz de Confusão para o conjunto de validação 3:
[[  0   0]
 [118   2]]

Métricas para o conjunto de validação 3:
Acurácia: 0.016666666666666666
Precisão: 1.0
Recall: 0.016666666666666666
F1-score: 0.03278688524590164

*******

1/1 [==============================] - 0s 37ms/step
Matriz de Confusão para o conjunto de validação 4:
[[ 1  0]
 [26  0]]

Métricas para o conjunto de validação 4:
Acurácia: 0.0370370370370370

Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


4/4 [==============================] - 0s 5ms/step
Matriz de Confusão para o conjunto de validação 6:
[[  1   0]
 [125   1]]

Métricas para o conjunto de validação 6:
Acurácia: 0.015748031496062992
Precisão: 1.0
Recall: 0.007936507936507936
F1-score: 0.015748031496062992

*******

5/5 [==============================] - 0s 4ms/step
Matriz de Confusão para o conjunto de validação 7:
[[  0   0]
 [133   3]]

Métricas para o conjunto de validação 7:
Acurácia: 0.022058823529411766
Precisão: 1.0
Recall: 0.022058823529411766
F1-score: 0.04316546762589929

*******

3/3 [==============================] - 0s 6ms/step
Matriz de Confusão para o conjunto de validação 8:
[[ 0  0]
 [66  1]]

Métricas para o conjunto de validação 8:
Acurácia: 0.014925373134328358
Precisão: 1.0
Recall: 0.014925373134328358
F1-score: 0.029411764705882353

*******

4/4 [==============================] - 0s 3ms/step
Matriz de Confusão para o conjunto de validação 9:
[[  0   0]
 [124   1]]

Métricas para o conjunto de valid

Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


3/3 [==============================] - 0s 8ms/step
Matriz de Confusão para o conjunto de validação 17:
[[ 0  0]
 [84  2]]

Métricas para o conjunto de validação 17:
Acurácia: 0.023255813953488372
Precisão: 1.0
Recall: 0.023255813953488372
F1-score: 0.04545454545454545

*******

12/12 [==============================] - 0s 1ms/step
Matriz de Confusão para o conjunto de validação 18:
[[ 52   0]
 [294  11]]

Métricas para o conjunto de validação 18:
Acurácia: 0.17647058823529413
Precisão: 1.0
Recall: 0.036065573770491806
F1-score: 0.06962025316455697

*******

5/5 [==============================] - 0s 4ms/step
Matriz de Confusão para o conjunto de validação 19:
[[  0   0]
 [131   8]]

Métricas para o conjunto de validação 19:
Acurácia: 0.05755395683453238
Precisão: 1.0
Recall: 0.05755395683453238
F1-score: 0.10884353741496598

*******

3/3 [==============================] - 0s 4ms/step
Matriz de Confusão para o conjunto de validação 20:
[[ 1  0]
 [91  0]]

Métricas para o conjunto de valid

...

3/3 [==============================] - 0s 5ms/step
Matriz de Confusão para o conjunto de teste 12:
[[ 0  0]
 [94  1]]

Métricas para o conjunto de teste 12:
Precisão: 1.0
Recall: 0.010526315789473684
F1-score: 0.020833333333333332

*******

5/5 [==============================] - 0s 2ms/step
Matriz de Confusão para o conjunto de teste 13:
[[  0   0]
 [130   6]]

Métricas para o conjunto de teste 13:
Precisão: 1.0
Recall: 0.04411764705882353
F1-score: 0.08450704225352113

*******

4/4 [==============================] - 0s 2ms/step
Matriz de Confusão para o conjunto de teste 14:
[[  0   0]
 [114   0]]

Métricas para o conjunto de teste 14:
Precisão: 0.0
Recall: 0.0
F1-score: 0.0

*******

2/2 [==============================] - 0s 9ms/step
Matriz de Confusão para o conjunto de teste 15:
[[ 0  0]
 [48  1]]

Métricas para o conjunto de teste 15:
Precisão: 1.0
Recall: 0.02040816326530612
F1-score: 0.039999999999999994

*******



Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


3/3 [==============================] - 0s 4ms/step
Matriz de Confusão para o conjunto de teste 16:
[[ 0  0]
 [71  0]]

Métricas para o conjunto de teste 16:
Precisão: 0.0
Recall: 0.0
F1-score: 0.0

*******

3/3 [==============================] - 0s 0s/step
Matriz de Confusão para o conjunto de teste 17:
[[ 1  0]
 [86  0]]

Métricas para o conjunto de teste 17:
Precisão: 0.0
Recall: 0.0
F1-score: 0.0

*******



Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


12/12 [==============================] - 0s 2ms/step
Matriz de Confusão para o conjunto de teste 18:
[[ 51   0]
 [303   3]]

Métricas para o conjunto de teste 18:
Precisão: 1.0
Recall: 0.00980392156862745
F1-score: 0.01941747572815534

*******

5/5 [==============================] - 0s 2ms/step
Matriz de Confusão para o conjunto de teste 19:
[[  0   0]
 [134   6]]

Métricas para o conjunto de teste 19:
Precisão: 1.0
Recall: 0.04285714285714286
F1-score: 0.0821917808219178

*******

3/3 [==============================] - 0s 0s/step
Matriz de Confusão para o conjunto de teste 20:
[[ 0  0]
 [92  1]]

Métricas para o conjunto de teste 20:
Precisão: 1.0
Recall: 0.010752688172043012
F1-score: 0.021276595744680854

*******

Média das Métricas:
Acurácia: 0.026344882539980262
Precisão: 0.7
Recall: 0.017854291585404473
F1-score: 0.03395606613441925


### Utilização dos conjuntos de NFT's Comuns como dados de Testes para fazer predição do modelo

In [22]:
# Fazer as previsões no conjunto de teste

# Definir um threshold para classificação de anomalias
threshold = 0.1  # Você pode ajustar este valor conforme necessário

for i in range(len(ts_nft_comum)):

    # Fazer as previsões no conjunto de teste
    test_data = ts_nft_comum[i]
    test_pred = autoencoder.predict(test_data)
    distances_test = tf.norm(test_data - test_pred, axis=1).numpy()

    # Fazer as previsões binárias no conjunto de teste usando o limiar
    test_pred_binary = [1 if dist > threshold else 0 for dist in distances_test]

    # Selecionar apenas a coluna com as classes de anomalia no conjunto de teste
    test_labels = test_data.iloc[:, 0].values
    # Transformar as classes de anomalia em 0 (não anomalia) e 1 (anomalia)
    test_labels_binary = [1 if label != 0 else 0 for label in test_labels]

    # Calcular métricas para o teste
    accuracy_test = accuracy_score(test_labels_binary, test_pred_binary)
    recall_test = recall_score(test_labels_binary, test_pred_binary)
    precision_test = precision_score(test_labels_binary, test_pred_binary)
    f1_score_test = f1_score(test_labels_binary, test_pred_binary)
    
    # Armazenar os valores das métricas nas listas
    accuracy_scores.append(accuracy_test)
    precision_scores.append(precision_test)
    recall_scores.append(recall_test)
    f1_scores.append(f1_score_test)

    # Calcular a matriz de confusão para o conjunto de teste
    confusion_test = confusion_matrix(test_labels_binary, test_pred_binary)

    # Verificar se exite algum registro classificado como analia na matriz de confusão - TP > 0
    tp_found = encontrar_tp(confusion_test)
    true_positive_anomalies.append(tp_found)
    
    # Imprimir a matriz de confusão
    print("Matriz de Confusão para o conjunto de teste: " + str(i+1) + ":")
    print(confusion_test) 
    
    # Imprimir as métricas
    print("\nMétricas para o conjunto de teste: " + str(i+1) + ":")
    print("Acurácia:", accuracy_test)
    print("Precisão:", precision_test)
    print("Recall:", recall_test)
    print("F1-score:", f1_score_test)
    print("\n*******\n")
    
# Calcular as médias das métricas após o loop
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
average_precision = sum(precision_scores) / len(precision_scores)
average_recall = sum(recall_scores) / len(recall_scores)
average_f1 = sum(f1_scores) / len(f1_scores)

# Armazenar médias das métricas para Heatmap
accuracy_scores_heatmap.append(average_accuracy)
precision_scores_heatmap.append(average_precision)
recall_scores_heatmap.append(average_recall)
f1_scores_heatmap.append(average_f1)

# Imprimir as médias das métricas
print("Média das Métricas:")
print("Acurácia:", average_accuracy)
print("Precisão:", average_precision)
print("Recall:", average_recall)
print("F1-score:", average_f1)


1/1 [==============================] - 0s 39ms/step
Matriz de Confusão para o conjunto de teste: 1:
[[ 5  0]
 [14  5]]

Métricas para o conjunto de teste: 1:
Acurácia: 0.4166666666666667
Precisão: 1.0
Recall: 0.2631578947368421
F1-score: 0.4166666666666667

*******

1/1 [==============================] - 0s 40ms/step
Matriz de Confusão para o conjunto de teste: 2:
[[0 1]
 [1 3]]

Métricas para o conjunto de teste: 2:
Acurácia: 0.6
Precisão: 0.75
Recall: 0.75
F1-score: 0.75

*******

1/1 [==============================] - 0s 31ms/step
Matriz de Confusão para o conjunto de teste: 3:
[[1 0]
 [2 7]]

Métricas para o conjunto de teste: 3:
Acurácia: 0.8
Precisão: 1.0
Recall: 0.7777777777777778
F1-score: 0.8750000000000001

*******

1/1 [==============================] - 0s 32ms/step
Matriz de Confusão para o conjunto de teste: 4:
[[ 1  0]
 [17  5]]

Métricas para o conjunto de teste: 4:
Acurácia: 0.2608695652173913
Precisão: 1.0
Recall: 0.22727272727272727
F1-score: 0.37037037037037035

****

Z:\Users\rogerio.zupo\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Matriz de Confusão para o conjunto de teste: 11:
[[ 2  0]
 [12  8]]

Métricas para o conjunto de teste: 11:
Acurácia: 0.45454545454545453
Precisão: 1.0
Recall: 0.4
F1-score: 0.5714285714285715

*******

1/1 [==============================] - 0s 27ms/step
Matriz de Confusão para o conjunto de teste: 12:
[[1 0]
 [5 6]]

Métricas para o conjunto de teste: 12:
Acurácia: 0.5833333333333334
Precisão: 1.0
Recall: 0.5454545454545454
F1-score: 0.7058823529411764

*******

1/1 [==============================] - 0s 37ms/step
Matriz de Confusão para o conjunto de teste: 13:
[[1 0]
 [2 5]]

Métricas para o conjunto de teste: 13:
Acurácia: 0.75
Precisão: 1.0
Recall: 0.7142857142857143
F1-score: 0.8333333333333333

*******

2/2 [==============================] - 0s 6ms/step
Matriz de Confusão para o conjunto de teste: 14:
[[ 1  0]
 [33  3]]

Métricas para o conjunto de teste: 14:
Acurácia: 0.10810810810810811
Precisão: 1.0
Recall: 0.08333333333333333
F1-score: 0.15384615384615385

*******

1/1 [=====